Let´s import the libriries and the datasets

In [9]:
import pandas as pd
import re

In [43]:
path='//content/drive/MyDrive/DataSets'
df=pd.read_csv(path+'/electricity_case.csv',sep=",",encoding='latin-1')
monthly_tariff=pd.read_csv(path+'/monthly_tariff.csv',sep=",",encoding='latin-1')
hourly_tariff=pd.read_csv(path+'/hourly_tariff.csv',sep=",",encoding='latin-1')

As you can see, is a messy dataset. Each row contain multiple information in one string, so the first part of the job is to break them down.  

In [16]:
df

,Raw
0,3 PM Mon 24th-Mar-2014___0.384 kwh
1,5AM 15-Ago-2014___1.201 kwh
2,__8PM Thu 20-Mar-2014____1.523 kwh
3,6PM 23rd-Abr-2014___0.424 kwh
4,_1AM Friday 19th-Dic-2014___0.209 kwh
...,...
8755,_1AM Fri 07th-Nov-2014_0.084 kwh
8756,_6AM 20-May-2014__1.027 kwh
8757,__2 AM Tuesday 8th-Abr-2014___0.052 kwh
8758,9 PM 27th-Ene-2014_1.428 kwh


In [7]:
# Let's create a copy of the dataset, just in case!
df_modified=df.copy()

In [89]:
#To clean this dataset, I'll use Regular expressions. So let's define a pattern.
pattern=re.compile(r'[\s*|_*]*(\d+\s?[AM|PM]+)\s*\w*\s*(\d+\w*-\w{3}-\d{4})[_]+(\d\.\d+)')

In [91]:
#Let's check how the patter works
df_modified['Raw'].str.findall(pattern)

0       [(3 PM, 24th-Mar-2014, 0.384)]
1           [(5AM, 5-Ago-2014, 1.201)]
2          [(8PM, 20-Mar-2014, 1.523)]
3         [(6PM, 3rd-Abr-2014, 0.424)]
4        [(1AM, 19th-Dic-2014, 0.209)]
                     ...              
8755     [(1AM, 07th-Nov-2014, 0.084)]
8756        [(6AM, 0-May-2014, 1.027)]
8757     [(2 AM, 8th-Abr-2014, 0.052)]
8758     [(9 PM, 7th-Ene-2014, 1.428)]
8759    [(12 PM, 5th-Ago-2014, 0.528)]
Name: Raw, Length: 8760, dtype: object

In [92]:
#It seems that the pattern worked for the Hour and the kwh consume.
#Let´s extract this data in new columns
df_modified[['Hour','Date','kwh']]=df_modified['Raw'].str.extract(pattern)


In [94]:
#We have to define the pattern for the date, and check how it works
patterndate=re.compile(r'(\d+\w*-\w{3}-\d{4})')
df_modified['Raw'].str.findall(patterndate)

0       [24th-Mar-2014]
1         [15-Ago-2014]
2         [20-Mar-2014]
3       [23rd-Abr-2014]
4       [19th-Dic-2014]
             ...       
8755    [07th-Nov-2014]
8756      [20-May-2014]
8757     [8th-Abr-2014]
8758    [27th-Ene-2014]
8759    [25th-Ago-2014]
Name: Raw, Length: 8760, dtype: object

In [105]:
#Now the patter for the date works! 
#Let's update the 'date' column and take a look at the dateset
df_modified['Date']=df_modified['Raw'].str.extract(patterndate)
df_modified

,Raw,Hour,Date,kwh
0,3 PM Mon 24th-Mar-2014___0.384 kwh,3PM,24th-Mar-2014,0.384
1,5AM 15-Ago-2014___1.201 kwh,5AM,15-Ago-2014,1.201
2,__8PM Thu 20-Mar-2014____1.523 kwh,8PM,20-Mar-2014,1.523
3,6PM 23rd-Abr-2014___0.424 kwh,6PM,23rd-Abr-2014,0.424
4,_1AM Friday 19th-Dic-2014___0.209 kwh,1AM,19th-Dic-2014,0.209
...,...,...,...,...
8755,_1AM Fri 07th-Nov-2014_0.084 kwh,1AM,07th-Nov-2014,0.084
8756,_6AM 20-May-2014__1.027 kwh,6AM,20-May-2014,1.027
8757,__2 AM Tuesday 8th-Abr-2014___0.052 kwh,2AM,8th-Abr-2014,0.052
8758,9 PM 27th-Ene-2014_1.428 kwh,9PM,27th-Ene-2014,1.428


In [106]:
#I need the Hour without any space, and the days without the termination th,rd,nd,st
df_modified['Hour']=df_modified.Hour.str.replace(' ', '')
df_modified['Date']=df_modified.Date.replace(to_replace=('th','st','rd','nd'), value='', regex=True)
df_modified.tail()

,Raw,Hour,Date,kwh
8755,_1AM Fri 07th-Nov-2014_0.084 kwh,1AM,07-Nov-2014,0.084
8756,_6AM 20-May-2014__1.027 kwh,6AM,20-May-2014,1.027
8757,__2 AM Tuesday 8th-Abr-2014___0.052 kwh,2AM,8-Abr-2014,0.052
8758,9 PM 27th-Ene-2014_1.428 kwh,9PM,27-Ene-2014,1.428
8759,_12 PM 25th-Ago-2014_0.528 kwh,12PM,25-Ago-2014,0.528


In [102]:
#Just to be sure, let's check if there is any NaN value
df_modified.isna().sum()

Raw      0
Hour     0
Date     0
kwh      0
Month    0
dtype: int64

The second part of the job is to join the consumption dataset with the monthly and hourly rates.

---

In [52]:
#Let's check the tariff tables first
print(monthly_tariff.head())
print(hourly_tariff.head())

  Month  Tariff
0   Ene    0.20
1   Feb    0.19
2   Mar    0.17
3   Abr    0.18
4   May    0.22
   Hour  Tariff
0  12AM    0.10
1   1AM    0.10
2   2AM    0.10
3   3AM    0.10
4   4AM    0.15


In [108]:
#I'll need to extract the months from the dataset in a new column, to merge it with the tariff table 
#let's define a pattern for the month to extract the month names
patternMonth=re.compile(r'(\w{3})')
df_modified['Month']=df_modified.Date.str.extract(patternMonth)

In [120]:
#Let´s join both table
df_modified=df_modified.merge(monthly_tariff,on='Month').merge(hourly_tariff,on='Hour')
df_modified.tail()

,Raw,Hour,Date,kwh,Month,Tariff_x,Tariff_y
8755,__11 AM Friday 3rd-Oct-2014_0.273 kwh,11AM,3-Oct-2014,0.273,Oct,0.15,0.26
8756,11AM Sun 19-Oct-2014_0.364 kwh,11AM,19-Oct-2014,0.364,Oct,0.15,0.26
8757,__11AM Sat 04-Oct-2014_0.41 kwh,11AM,04-Oct-2014,0.41,Oct,0.15,0.26
8758,__11AM Thursday 09-Oct-2014__0.243 kwh,11AM,09-Oct-2014,0.243,Oct,0.15,0.26
8759,__11 AM 27-Oct-2014__0.258kwh,11AM,27-Oct-2014,0.258,Oct,0.15,0.26


In [122]:
#Let's rename the tariff columns
df_modified.rename(columns={'Tariff_x':'Monthly_tariff','Tariff_y':'Hourly_tariff'},inplace=True)

In [125]:
#I need to add one extra column, for the fixed tariff 0.21
#df_modified['Fixed_tariff']=0.21
df_modified.tail()

,Raw,Hour,Date,kwh,Month,Monthly_tariff,Hourly_tariff,Fixed_tariff
8755,__11 AM Friday 3rd-Oct-2014_0.273 kwh,11AM,3-Oct-2014,0.273,Oct,0.15,0.26,0.21
8756,11AM Sun 19-Oct-2014_0.364 kwh,11AM,19-Oct-2014,0.364,Oct,0.15,0.26,0.21
8757,__11AM Sat 04-Oct-2014_0.41 kwh,11AM,04-Oct-2014,0.41,Oct,0.15,0.26,0.21
8758,__11AM Thursday 09-Oct-2014__0.243 kwh,11AM,09-Oct-2014,0.243,Oct,0.15,0.26,0.21
8759,__11 AM 27-Oct-2014__0.258kwh,11AM,27-Oct-2014,0.258,Oct,0.15,0.26,0.21


The last part is to calculate the cost of electricity consumed, for each rate

___

In [135]:
#the values in the column 'kwh' are objetc type, so, first we need to converte it to float type
df_modified['kwh']=df_modified['kwh'].astype(float)

In [138]:
#Now we can calculate the cost for each tariff
df_modified['Cost_monthly_tariff']=df_modified.kwh*df_modified.Monthly_tariff
df_modified['Cost_hourly_tariff']=df_modified.kwh*df_modified.Hourly_tariff
df_modified['Cost_fixed_tariff']=df_modified.kwh*df_modified.Fixed_tariff
df_modified.tail()

,Raw,Hour,Date,kwh,Month,Monthly_tariff,Hourly_tariff,Fixed_tariff,Cost_monthly_tariff,Cost_hourly_tariff,Cost_fixed_tariff
8755,__11 AM Friday 3rd-Oct-2014_0.273 kwh,11AM,3-Oct-2014,0.273,Oct,0.15,0.26,0.21,0.04095,0.07098,0.05733
8756,11AM Sun 19-Oct-2014_0.364 kwh,11AM,19-Oct-2014,0.364,Oct,0.15,0.26,0.21,0.05460,0.09464,0.07644
8757,__11AM Sat 04-Oct-2014_0.41 kwh,11AM,04-Oct-2014,0.410,Oct,0.15,0.26,0.21,0.06150,0.10660,0.08610
8758,__11AM Thursday 09-Oct-2014__0.243 kwh,11AM,09-Oct-2014,0.243,Oct,0.15,0.26,0.21,0.03645,0.06318,0.05103
8759,__11 AM 27-Oct-2014__0.258kwh,11AM,27-Oct-2014,0.258,Oct,0.15,0.26,0.21,0.03870,0.06708,0.05418


In [139]:
#There is no missing values, so it seems that the process went just perfect!
df_modified.isna().sum()

Raw                    0
Hour                   0
Date                   0
kwh                    0
Month                  0
Monthly_tariff         0
Hourly_tariff          0
Fixed_tariff           0
Cost_monthly_tariff    0
Cost_hourly_tariff     0
Cost_fixed_tariff      0
dtype: int64

In [140]:
#Finally, let's export this as .csv file
df_modified.to_csv('/Electricity_tariff_output.csv', index=False)
!cp /Electricity_tariff_output.csv "drive/My Drive/"